In [1]:
import ipyparallel as ipp
from mpi4py import MPI
c = ipp.Client(profile='mpi')

In [2]:
%%px --no-stream --group-outputs=engine

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [3]:
%%px --no-stream --group-outputs=engine
import sys
import os
sys.path.insert(0, os.path.abspath("/home/hajta2/study/sonar-FWI/cli"))
from simulation.sonar import Sonar
from simulation.utils import CircleBottom, EllipsisBottom, run_beam, positions_line,gen_velocity_profile
from simulation.sources import GaborSource
from simulation.plotting import plot_velocity

from examples.seismic import Receiver

import matplotlib.pyplot as plt
import numpy as np

from devito import configuration
configuration['mpi'] = True
configuration['language'] = 'C'

In [4]:
%%px

domain_size = (6, 3)
radius = 2.8
v_env = 1.5
source_distance = 0.002
ns = 128
cy = (ns - 1) / 2 * source_distance
f0 = 50
spatial_dist = round(v_env / f0 / 3, 3) / 2
domain_dims = (
    round(domain_size[0] / spatial_dist),
    round(domain_size[1] / spatial_dist),
)

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [5]:
%%px --no-stream --group-output=engine

if rank == 0:
    vp = gen_velocity_profile(
        EllipsisBottom(True), domain_dims, spatial_dist, v_water=v_env
    )
else:
    vp = None
vp = comm.bcast(vp, root=0)

[stderr:0] 
51it [00:00, 324.79it/s]


In [6]:
%%px --no-stream --group-outputs=engine

sonar = Sonar(domain_size, f0, v_env, vp, source_distance=source_distance, ns=ns, spatial_dist=spatial_dist)

[stderr:0] 
Operator `initdamp` ran in 0.02 s
[stderr:1] 
Operator `initdamp` ran in 0.02 s


In [11]:
%%px --no-stream --group-outputs=engine

sonar.set_source()

In [16]:
%%px --no-stream --group-outputs=engine

sonar.set_source()
sonar.finalize()

%px: 100%|██████████| 2/2 [00:01<00:00,  1.39tasks/s]


In [23]:
%%px

sonar.src.coordinates.data

Out[1:22]: 
Data([[3.00100787, 0.129     ],
      [3.00302362, 0.129     ],
      [3.00503937, 0.129     ],
      [3.00705512, 0.129     ],
      [3.00907087, 0.129     ],
      [3.01108661, 0.129     ],
      [3.01310236, 0.129     ],
      [3.01511811, 0.129     ],
      [3.01713386, 0.129     ],
      [3.01914961, 0.129     ],
      [3.02116535, 0.129     ],
      [3.0231811 , 0.129     ],
      [3.02519685, 0.129     ],
      [3.0272126 , 0.129     ],
      [3.02922835, 0.129     ],
      [3.03124409, 0.129     ],
      [3.03325984, 0.129     ],
      [3.03527559, 0.129     ],
      [3.03729134, 0.129     ],
      [3.03930709, 0.129     ],
      [3.04132283, 0.129     ],
      [3.04333858, 0.129     ],
      [3.04535433, 0.129     ],
      [3.04737008, 0.129     ],
      [3.04938583, 0.129     ],
      [3.05140157, 0.129     ],
      [3.05341732, 0.129     ],
      [3.05543307, 0.129     ],
      [3.05744882, 0.129     ],
      [3.05946457, 0.129     ],
      [3.06148031, 0.129    

Out[0:22]: 
Data([[2.872     , 0.129     ],
      [2.87401575, 0.129     ],
      [2.8760315 , 0.129     ],
      [2.87804724, 0.129     ],
      [2.88006299, 0.129     ],
      [2.88207874, 0.129     ],
      [2.88409449, 0.129     ],
      [2.88611024, 0.129     ],
      [2.88812598, 0.129     ],
      [2.89014173, 0.129     ],
      [2.89215748, 0.129     ],
      [2.89417323, 0.129     ],
      [2.89618898, 0.129     ],
      [2.89820472, 0.129     ],
      [2.90022047, 0.129     ],
      [2.90223622, 0.129     ],
      [2.90425197, 0.129     ],
      [2.90626772, 0.129     ],
      [2.90828346, 0.129     ],
      [2.91029921, 0.129     ],
      [2.91231496, 0.129     ],
      [2.91433071, 0.129     ],
      [2.91634646, 0.129     ],
      [2.9183622 , 0.129     ],
      [2.92037795, 0.129     ],
      [2.9223937 , 0.129     ],
      [2.92440945, 0.129     ],
      [2.9264252 , 0.129     ],
      [2.92844094, 0.129     ],
      [2.93045669, 0.129     ],
      [2.93247244, 0.129    

In [17]:
%%px

if rank == 0:
    gathered_data = np.empty((sonar.src.data.shape[0], sonar.src.data.shape[1] * size), dtype=np.float64)
else:
    gathered_data = None

comm.Gather(sonar.src.data, gathered_data, root=0)

In [22]:
%%px

batch = ns // size
for i in range(rank * batch, (rank + 1) * batch):
    # print(sonar.src.data[:, i])
    print(np.count_nonzero(sonar.src.data[:, i]))

[stdout:0] 6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819


[stdout:1] 6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819
6819


In [24]:
%%px

src = sonar.src
alpha = 45
dt = sonar.model.critical_dt
ns = src.coordinates.data.shape[0]
batch = ns // size

if alpha <= 90:
    max_latency = (
        np.cos(np.deg2rad(alpha)) * ((ns - 1) * source_distance / v_env) / dt
    )
elif alpha > 90:
    max_latency = np.cos(np.deg2rad(alpha)) * (source_distance / v_env) / dt
for i in range(rank * batch, (rank + 1) * batch):
    latency = -np.cos(np.deg2rad(alpha)) * (i * source_distance / v_env)
    src.data[:, i] = np.roll(src.data[:, i], int(latency / dt + max_latency))

[0:execute]
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[23], line 15
     13 for i in range(rank * batch, (rank + 1) * batch):
     14     latency = -np.cos(np.deg2rad(alpha)) * (i * source_distance / v_env)
---> 15     src.data[:, i] = np.roll(src.data[:, i], int(latency / dt + max_latency))

File <__array_function__ internals>:200, in roll(*args, **kwargs)

File ~/.local/lib/python3.11/site-packages/numpy/core/numeric.py:1226, in roll(a, shift, axis)
   1224 a = asanyarray(a)
   1225 if axis is None:
-> 1226     return roll(a.ravel(), shift, 0).reshape(a.shape)
   1228 else:
   1229     axis = normalize_axis_tuple(axis, a.ndim, allow_duplicate=True)

File <__array_function__ internals>:200, in roll(*args, **kwargs)

File ~/.local/lib/python3.11/site-packages/numpy/core/numeric.py:1249, in roll(a, shift, axis)
   1247 for indices in itertools.product(*rolls):
   1248    

AlreadyDisplayedError: 2 errors